In [21]:
# Installing required libraries
! pip install sklearn
! pip install pandas
! pip install sweetviz
! pip install imbalanced-learn
! pip install xgboost

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/inder/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/inder/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/inder/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/inder/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/inder/anaconda3/lib

In [26]:
# Import the libraries
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import sweetviz as sv
from sklearn import metrics
from scipy import stats
import imblearn
from sklearn import preprocessing
from imblearn.under_sampling import CondensedNearestNeighbour
import xgboost as xgb

In [8]:
# Fetch the data
data_train = pd.read_csv('Train.csv', sep=',',header=0)
data_eval = pd.read_csv('Eval.csv', sep=',',header=0)
data_train.dropna(how='all',inplace=True)
data_train = data_train[(np.abs(stats.zscore(data_train)) < 5).all(axis=1)]
feat =  data_train.iloc[:,:-1]
label = data_train['class_col']
print(pd.unique(label)) # Binary Classification

[0. 1.]


In [9]:
#Normalization
min_max_scaler = preprocessing.Normalizer()
x_scaled = min_max_scaler.fit_transform(feat.values)

x_scaled

array([[ 3.85893870e-01, -2.17003331e-01,  7.16110992e-01, ...,
         0.00000000e+00,  2.38703664e-01,  4.77407328e-01],
       [-1.66506942e-02,  8.52122884e-01,  0.00000000e+00, ...,
        -2.84040961e-01,  0.00000000e+00,  0.00000000e+00],
       [ 7.58127547e-01,  1.07798948e-01,  0.00000000e+00, ...,
        -2.37157685e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 3.92478379e-02,  8.42800248e-02,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  9.27080272e-01],
       [ 2.00566653e-05, -1.14022133e-03,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.48889598e-02,  2.20756232e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [10]:
# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(
    x_scaled, label,test_size =0.30,shuffle=True)

The next step was to do under-sampling to balance the training data to have better representation of both the classes, the technique I am using is condensed nearest neighbours, unfortunately due to the lack of computing power I was not able to complete the test for this run.

In [11]:

# # define the undersampling method
# undersample = CondensedNearestNeighbour(n_neighbors=1)
# # transform the dataset
# X_train, y_train = undersample.fit_resample(X_train, y_train)

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

over = RandomOverSampler(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.8)
# first performing oversampling to minority class
X_over, y_over = over.fit_resample(X_train, y_train)
# print(f"Oversampled: {Counter(y_over)}")
# Oversampled: Counter({0: 9844, 1: 4922})
# now to comine under sampling 
# X_combined_sampling, y_combined_sampling 
X_train, y_train = under.fit_resample(X_over, y_over)

In [16]:
# zoo = pd.DataFrame(y_train)
# print(pd.unique(zoo))

In [13]:
feature = pd.DataFrame(X_train)

# EDA of the training file
eda_report = sv.analyze(feature)

# Saving results to HTML file
eda_report.show_html('eda_report_sampling.html')

                                             |          | [  0%]   00:00 -> (? left)

Report eda_report_sampling.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [17]:
# KNN-classification
KNN = KNeighborsClassifier(n_neighbors=10).fit(X_train, y_train)

KNN_pred = KNN.predict(X_test)
print("KNN model accuracy(in %):",metrics.accuracy_score(y_test, KNN_pred)*100)
print("KNN model f1-score(in %):",metrics.f1_score(y_test, KNN_pred)*100)

KNN model accuracy(in %): 78.01301284428412
KNN model f1-score(in %): 27.58161225849434


In [18]:
# NaiveBayes Classification
gnb = GaussianNB().fit(X_train, y_train) 
gnb_pred = gnb.predict(X_test)
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, gnb_pred)*100)
print("Gaussian Naive Bayes model f1-score(in %):",metrics.f1_score(y_test, gnb_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 87.54677544415603
Gaussian Naive Bayes model f1-score(in %): 33.87039026136771


In [19]:
# Logistic Regression

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
LR_pred = LR.predict(X_test)
print("LR model accuracy(in %):",metrics.accuracy_score(y_test, LR_pred)*100)
print("LR  model f1-score(in %):",metrics.f1_score(y_test, LR_pred)*100)

LR model accuracy(in %): 82.86417422378047
LR  model f1-score(in %): 30.41752224503765


In [ ]:
# Support Vector Machine
# SVM = svm.LinearSVC().fit(X_train, y_train)
SVM = svm.SVC(probability=True).fit(X_train, y_train)
SVM_pred = SVM.predict(X_test)
print("SVM model accuracy(in %):",metrics.accuracy_score(y_test, SVM_pred)*100)
print("SVM model f1-score(in %):",metrics.f1_score(y_test, SVM_pred)*100)

In [20]:
#Multi-Layer Perceptron
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1).fit(X_train, y_train)
NN_pred = NN.predict(X_test)
print("MLP model accuracy(in %):",metrics.accuracy_score(y_test, NN_pred)*100)
print("MLP model f1-score(in %):",metrics.f1_score(y_test, NN_pred)*100)

MLP model accuracy(in %): 82.83720459832115
MLP model f1-score(in %): 30.51726491060462


/home/inder/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [27]:
# XGBoost
XGB =  xgb.XGBClassifier(max_depth= 3, n_estimators=100).fit(X_train, y_train)
xgb_pred = XGB.predict(X_test)
print("MLP model accuracy(in %):",metrics.accuracy_score(y_test, xgb_pred)*100)
print("MLP model f1-score(in %):",metrics.f1_score(y_test, xgb_pred)*100)

MLP model accuracy(in %): 82.85406061423322
MLP model f1-score(in %): 34.55995882655687


In [32]:
data_eval.head()

,variable_0,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9
0,-0.235544,0.090909,0,-0.408420,0.759402,0.000000,0,-0.5,0,0
1,-0.273905,0.681818,1,-0.526768,0.132906,-0.666667,0,-0.5,0,0
2,0.258914,0.045455,0,0.035590,-0.097863,0.333333,0,0.0,0,2
3,-0.291358,0.227273,0,-0.520809,0.032906,-0.500000,0,0.0,0,0
4,1.060935,-0.681818,0,-0.210981,-0.232051,-0.833333,0,-0.5,0,4


In [34]:
model_prob = XGB.predict_proba(data_eval.values)
model_prob

array([[0.9372817 , 0.06271828],
       [0.93185216, 0.06814784],
       [0.7664131 , 0.23358689],
       ...,
       [0.83118546, 0.16881451],
       [0.847976  , 0.15202397],
       [0.7468232 , 0.2531768 ]], dtype=float32)

In [35]:
prob = list(model_prob[:,-1])
class1_prob = [round(x,3) for x in prob]



In [36]:
output = pd.DataFrame(columns=['row_id', 'yhat'])
output['yhat'] = class1_prob
output['row_id'] = [x for x in range(len(class1_prob))]

In [38]:
output.to_csv('output.csv',index=False)